https://adventofcode.com/2015/day/22

In [ ]:
test_player_stats = {
    'Hit Points': 10,
    'Mana': 250,
    'Armor': 0
}

test_boss_stats = {
    'Hit Points': 13,
    'Damage': 8
}

In [ ]:
player_stats = {
    'Hit Points': 50,
    'Armor': 0,
    'Mana': 500
}

boss_stats = {
    'Hit Points': 55,
    'Damage': 8}

In [ ]:
spell_book = r'''Magic Missile costs 53 mana. It instantly does 4 damage.
Drain costs 73 mana. It instantly does 2 damage and heals you for 2 hit points.
Shield costs 113 mana. It starts an effect that lasts for 6 turns. While it is active, your armor is increased by 7.
Poison costs 173 mana. It starts an effect that lasts for 6 turns. At the start of each turn while it is active, it deals the boss 3 damage.
Recharge costs 229 mana. It starts an effect that lasts for 5 turns. At the start of each turn while it is active, it gives you 101 new mana.'''

In [ ]:
import re

regex = re.compile('(?P<spell_name>[A-Za-z]+? ?[A-Za-z]+) costs (?P<cost>[0-9]+) mana.*')
regex2 = re.compile('(?P<spell_name>[A-Za-z]+) costs (?P<cost>[0-9]+) mana. It starts an effect that lasts for (?P<len>[0-9]+) turns.*(?P<size>[0-9]+)[^0-9].*')

In [ ]:
spell_dict = dict()

for lineno, line in enumerate(spell_book.split('\n')):
  if lineno < 2:
     name, cost = regex.fullmatch(line).group(1, 2)
     spell_dict[name] = [cost]
  else:
    name, cost, len, size = regex2.fullmatch(line).group(1, 2, 3, 4)
    print(spell_dict)
    spell_dict[name] = [cost, len, size]

spell_dict

{'Magic Missile': ['53'], 'Drain': ['73']}
{'Magic Missile': ['53'], 'Drain': ['73'], 'Shield': ['113', '6', '7']}
{'Magic Missile': ['53'], 'Drain': ['73'], 'Shield': ['113', '6', '7'], 'Poison': ['173', '6', '3']}


{'Drain': ['73'],
 'Magic Missile': ['53'],
 'Poison': ['173', '6', '3'],
 'Recharge': ['229', '5', '1'],
 'Shield': ['113', '6', '7']}

In [ ]:
import copy

class GameState:

  def __init__(self, p_stats, b_stats):
    self.p_stats = copy.deepcopy(p_stats)
    self.b_stats = copy.deepcopy(b_stats)
    self.p = 0 # effects booleans
    self.r = 0
    self.s = 0
    self.total_mana_spent = 0
    self.winner = None
    self.log = "\n-- Player turn --\n"
    
    self.log += "- Player stats: " + str(self.p_stats) + "\n"
    self.log += f"- Boss has {self.b_stats['Hit Points']} hit points\n"

  def cast_spell(self, spell):
    if self.winner is None:
      self.log += f"Player casts {spell} \n"
      getattr(self, '_' + spell.lower().replace(' ', '_'))()
      self._check_for_winner()

      if self.winner is None:
        self.log += "\n-- Boss turn --\n"
        self.log += "- Player stats: " + str(self.p_stats) + "\n"
        self.log += f"- Boss has {self.b_stats['Hit Points']} hit points\n"
        
        self._apply_effects()
        self._check_for_winner()

        if self.winner is None:
          self._take_boss_turn()
          self._check_for_winner()

          if self.winner is None:
            self.log += "\n-- Player turn --\n"
            self.log += "- Player stats: " + str(self.p_stats) + "\n"
            self.log += f"- Boss has {self.b_stats['Hit Points']} hit points\n"

            self._apply_effects()
            self._check_for_winner()

  def _check_for_winner(self):
    if self.winner is None:
      if self.p_stats['Hit Points'] <= 0:
        self.winner = 'Boss'
        self.log += 'Boss wins! :(\n'
      elif self.b_stats['Hit Points'] <= 0:
        self.winner = 'Player'
        self.log += 'Player wins!\n'

  def show_game(self):
    if self.winner is not None:
      print('Winner:', self.winner)
    print(' === ')
    print('Player: ', self.p_stats)
    print('Boss: ', self.b_stats)
    print(f'Posion: {self.p} Recharge: {self.r} Shield: {self.s}')
    print('Total Mana Spent: ', self.total_mana_spent)
    print(' === ')

  def _take_boss_turn(self):
    damage = max(1, self.b_stats['Damage']-self.p_stats['Armor'])
    self.p_stats['Hit Points'] -= damage
    self.log += f'Boss attacks for {damage} damage! \n'

  def _apply_effects(self):
    if self.s == 0:
      self.p_stats['Armor'] = 0
    if self.s > 0:
      self.p_stats['Armor'] = 8
      self.s -= 1
      self.log += f"Shield's timer is now {self.s}.\n"
    if self.p > 0:
      self.b_stats['Hit Points'] -= 3
      self.p -= 1
      self.log += f"Poison deals 3 damage; its timer is now {self.p}.\n"
    if self.r > 0:
      self.p_stats['Mana'] += 101
      self.r -= 1
      self.log += f"Recharge provides 101 mana; its timer is now {self.r}.\n"

  def _try_spend_mana(self, amount):
    if amount > self.p_stats['Mana']:
      self.winner = "Boss"
      self.log += 'failed spell!! >:/\n'
      return False
    else:
      self.total_mana_spent += amount
      self.p_stats['Mana'] -= amount
      return True

  def _drain(self):
    if self._try_spend_mana(73):
      self.p_stats['Hit Points'] += 2
      self.b_stats['Hit Points'] -= 2

  def _magic_missile(self):
    if self._try_spend_mana(53):
      self.b_stats['Hit Points'] -= 4
  
  def _poison(self):
    if self.p > 0:
      self.winner = 'Boss'
      self.log += "Can't cast twice, bro. \n"
    if self._try_spend_mana(173):
      self.p = 6

  def _recharge(self):
    if self.r > 0:
      self.winner = 'Boss'
      self.log += "Can't cast twice, bro. \n"
    if self._try_spend_mana(229):
      self.r = 5

  def _shield(self):
    if self.s > 0:
      self.winner = 'Boss'
      self.log += "Can't cast twice, bro. \n"
    if self._try_spend_mana(113):
      self.s = 6

In [ ]:
game = GameState(test_player_stats, test_boss_stats)

for spell in ('Poison', 'Magic Missile'):
  game.cast_spell(spell)

print(game.log)


-- Player turn --
- Player stats: {'Hit Points': 10, 'Mana': 250, 'Armor': 0}
- Boss has 13 hit points
Player casts Poison 

-- Boss turn --
- Player stats: {'Hit Points': 10, 'Mana': 77, 'Armor': 0}
- Boss has 13 hit points
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 damage! 

-- Player turn --
- Player stats: {'Hit Points': 2, 'Mana': 77, 'Armor': 0}
- Boss has 10 hit points
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile 

-- Boss turn --
- Player stats: {'Hit Points': 2, 'Mana': 24, 'Armor': 0}
- Boss has 3 hit points
Poison deals 3 damage; its timer is now 3.
Player wins!



In [ ]:
import numpy as np

def minimum_mana_h(game, early_stop=np.inf):
  game.log
  if game.winner == 'Boss':
    return np.inf
  elif game.winner == 'Player':
    return game.total_mana_spent
  best_mana_so_far = early_stop
  for spell in ('Magic Missile', 'Poison', 'Shield', 'Drain', 'Recharge'):
    new_game = copy.deepcopy(game)
    new_game.cast_spell(spell)
    if new_game.total_mana_spent > best_mana_so_far:
      continue
    m = minimum_mana_h(new_game, early_stop=best_mana_so_far)
    if m < best_mana_so_far:
      best_mana_so_far = m
  return best_mana_so_far

def minimum_mana(player_stats, boss_stats):
  game = GameState(player_stats, boss_stats)
  return minimum_mana_h(game)

minimum_mana(player_stats, boss_stats)

953

https://adventofcode.com/2015/day/22#part2

In [ ]:
import copy

class GameStateHard(GameState):

  def __init__(self, p_stats, b_stats):
    super().__init__(p_stats, b_stats)
    self.log += 'Player lost a point because HARD'
    self.p_stats['Hit Points'] -= 1

  def cast_spell(self, spell):
    if self.winner is None:
      self.log += f"Player casts {spell} \n"
      getattr(self, '_' + spell.lower().replace(' ', '_'))()
      self._check_for_winner()

      if self.winner is None:
        self.log += "\n-- Boss turn --\n"
        self.log += "- Player stats: " + str(self.p_stats) + "\n"
        self.log += f"- Boss has {self.b_stats['Hit Points']} hit points\n"
        
        self._apply_effects()
        self._check_for_winner()

        if self.winner is None:
          self._take_boss_turn()
          self._check_for_winner()

          if self.winner is None:
            self.log += "\n-- Player turn --\n"
            self.log += "- Player stats: " + str(self.p_stats) + "\n"
            self.log += 'Player lost a point because HARD'
            self.p_stats['Hit Points'] -= 1
            self._check_for_winner()
            if self.winner is None:
              self.log += f"- Boss has {self.b_stats['Hit Points']} hit points\n"
              self._apply_effects()
              self._check_for_winner()

In [ ]:
import numpy as np

def minimum_mana_h(game, early_stop=np.inf):
  game.log
  if game.winner == 'Boss':
    return np.inf
  elif game.winner == 'Player':
    return game.total_mana_spent
  best_mana_so_far = early_stop
  for spell in ('Magic Missile', 'Poison', 'Shield', 'Drain', 'Recharge'):
    new_game = copy.deepcopy(game)
    new_game.cast_spell(spell)
    if new_game.total_mana_spent > best_mana_so_far:
      continue
    m = minimum_mana_h(new_game, early_stop=best_mana_so_far)
    if m < best_mana_so_far:
      best_mana_so_far = m
  return best_mana_so_far

def minimum_mana_hard(player_stats, boss_stats):
  game = GameStateHard(player_stats, boss_stats)
  return minimum_mana_h(game)

minimum_mana_hard(player_stats, boss_stats)

1289